# Use Case
##### Warehouse use cases focus on fast queries that retrieve aggregated data by applying desired filters

# Data Warehouse Modeling techniques
* Industry-specific domain models such as OMG models 
  * Example: Object Management Group
  * (https://www.omg.org/industries/index.htm)
* Kimball
  * identifis the key business processes and the key business questions that needs to answer
  * key dimensions, like customer and product, that are shared across the different facts will be built once & reused
* Inmon
  * Identifies the key subject areas, and key entities the business operates with and cares about
  * Build data marts specific for departments/LOB
  * Data warehouse is the only source of data for the different data marts
* Data Vault methodologies
  * store raw data as-is without applying business rules
  * (https://datavaultalliance.com/)
  * has three types of entities: 
    * hubs: hold all unique business keys of a subject
    * links: track all relationships between hubs (join keys)
    * satellites: hold any attributes related to a link or hub and update them as they change

# Cleanup

In [0]:
# Clean prior run data files
dbutils.fs.rm('/tmp/ch-7/', True)

# Drop & recreate database
spark.sql("DROP DATABASE IF EXISTS ch_7 CASCADE")
spark.sql("CREATE DATABASE ch_7 ")
spark.sql("USE ch_7")

Out[1]: DataFrame[]

# Identity Columns

In [0]:
%sql
CREATE TABLE IF NOT EXISTS Identity_tbl (
  pKey bigint GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
  name string
);
insert into Identity_tbl (name) values ('a'),('b'),('c');
select * from Identity_tbl;

pKey,name
1,a
3,c
2,b


In [0]:
%sql
describe history Identity_tbl;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2022-08-04T00:08:51.000+0000,6490153397734611,anindita.mahapatra@databricks.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1322756337600463),0803-022207-5zm6vs41,0,WriteSerializable,true,"Map(numFiles -> 3, numOutputRows -> 3, numOutputBytes -> 2457)",null,Databricks-Runtime/11.1.x-scala2.12
0,2022-08-04T00:08:48.000+0000,6490153397734611,anindita.mahapatra@databricks.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1322756337600463),0803-022207-5zm6vs41,null,WriteSerializable,true,Map(),null,Databricks-Runtime/11.1.x-scala2.12


# Restore Delta Table
* It is more efficient using REPLACE instead of dropping and re-creating Delta Lake tables

In [0]:
%sql
Restore Table Identity_tbl to version as of 0;

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
0,0,3,0,2457,0


# Replace Table

In [0]:
%sql
CREATE or REPLACE TABLE Identity_tbl (
  pKey bigint GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
  name string
);

In [0]:
%sql
select * from Identity_tbl

pKey,name


# Primary Key and Foreign Key Constraints
* tested against DBR 11.1
* FK/PK contraints are informational only (and can be leveraged in reporting tools etc) but the data is not enforced.
* Table constraints are not supported with Hive Metastore

%sql
-- Create a table with a primary key
CREATE TABLE persons(
                     first_name STRING NOT NULL, 
                     last_name STRING NOT NULL, 
                     nickname STRING,
                     CONSTRAINT persons_pk PRIMARY KEY(first_name, last_name)
                    )
USING DELTA
LOCATION '/tmp/ch-5/pk';

-- create a table with a foreign key
CREATE TABLE pets(name STRING, 
                  owner_first_name STRING, 
                  owner_last_name STRING,
                  CONSTRAINT pets_persons_fk FOREIGN KEY (owner_first_name, owner_last_name) REFERENCES persons
                 )
USING DELTA
LOCATION '/tmp/ch-5/pk';

# Example Implementation

## Dimensional Modeling with Star Schema
* Efficiently stores data, 
* Maintains history and 
* Updates data by reducing the duplication of repetitive business definitions

In [0]:
%sql
DROP DATABASE IF EXISTS ch_7 CASCADE;
CREATE DATABASE ch_7;
USE ch_7;

## Create your fact and dimension Delta tables
* Surrogate Keys
  * A surrogate key is a type of primary key (non-natural) that helps to identify each record uniquely
* PK/FK constraint

#### Fact table

In [0]:
%sql
CREATE TABLE transaction ( 
  customer_id BIGINT, 
  product_sku STRING, 
  tx_Date TIMESTAMP, 
  units INT,
  sale_amount FLOAT
)
USING DELTA;

#### Dimension tables

In [0]:
%sql
CREATE TABLE product ( 
  product_sku STRING, 
  product_name STRING, 
  category STRING, 
  price FLOAT
)
USING DELTA;

In [0]:
%sql
CREATE TABLE customer ( 
  customer_id BIGINT not null GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
  customer_name STRING, 
  address STRING, 
  zip STRING, 
  status BOOLEAN
)
USING DELTA;

## Optimize file size for fast file pruning
* Data skipping can help with file pruning aand partition pruning 
* So what is Godilock zone for ideal data file size? A good file size range is 32-128MB 
* ALTER TABLE (database).(table) SET TBLPROPERTIES (delta.targetFileSize=33554432)

In [0]:
%sql
ALTER TABLE transaction SET TBLPROPERTIES (delta.targetFileSize=33554432)

## Create a Z-Order on fact tables
* ZORDER BY (LARGEST_DIM_FK, NEXT_LARGEST_DIM_FK, ...)

In [0]:
%sql
OPTIMIZE transaction
ZORDER BY (product_sku, customer_id);

path,metrics
dbfs:/user/hive/warehouse/ch_7.db/transaction,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(0, 0), 0, List(0, 0), 0, null), 0, 0, 0, false, 0, 0, 1659571755726, 1659571757248)"


## Create Z-Orders on dimension key fields and popular predicates
* ZORDER BY (BIG_DIM_PK, LIKELY_FIELD_1, LIKELY_FIELD_2)

In [0]:
%sql
OPTIMIZE product
ZORDER BY (product_sku, category, price);

path,metrics
dbfs:/user/hive/warehouse/ch_7.db/product,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(0, 0), 0, List(0, 0), 0, null), 0, 0, 0, false, 0, 0, 1659571758128, 1659571758944)"


In [0]:
%sql
OPTIMIZE customer
ZORDER BY (customer_id, zip);

path,metrics
dbfs:/user/hive/warehouse/ch_7.db/customer,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(0, 0), 0, List(0, 0), 0, null), 0, 0, 0, false, 0, 0, 1659571759784, 1659571760613)"


## Analyze Table to gather statistics for AQE Optimizer
* Adaptive Query Execution
* ANALYZE TABLE MY_BIG_DIM COMPUTE STATISTICS FOR ALL COLUMNS

In [0]:
%sql
ANALYZE TABLE transaction COMPUTE STATISTICS FOR ALL COLUMNS;
ANALYZE TABLE product COMPUTE STATISTICS FOR ALL COLUMNS;
ANALYZE TABLE customer COMPUTE STATISTICS FOR ALL COLUMNS;

# Data Organization
* Bronze layer is the landing area here data is retained as is
* Silver layer is more vault like where transformations are applied
* Gold layer is the presentation layer which is read optimized & where star/snowflake schema use aggregated data oof fact tables

# BI tool connectivity
* Power BI: https://github.com/delta-io/connectors/tree/master/powerbi